<a href="https://colab.research.google.com/github/LeeMinJun0102/Car_Price_Predict/blob/main/ML_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.font_manager as fm
import ast

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.svm import SVR
from sklearn.tree import plot_tree, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, cross_validate, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score

from scipy.stats import randint, uniform, loguniform

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_excel('Final.xlsx')
df.info()

In [ ]:
df.head()

# X / y

In [ ]:
X = df[['manufacturer_grouping',
        'car_type_grouping',
        'mileage',
        'color_grouping',
        'fuel_grouping',
        'transmission_grouping',
        'purchase_accident',
        'sales_channel_grouping',
        'purchase_channel_1_grouping',
        'new_car_price',
        'usedyear'
       ]]

y = df['residual_value_rate']
X.head()

# train / test 분할

In [ ]:
# train/test 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# LightGBM

## 전처리

LabelEncoding 사용: LightGBM은 범주형 데이터를 직접 처리할 수 있어 OneHotEncoding보다 효율적입니다.

In [ ]:
categorical_features = ['manufacturer_grouping', 'car_type_grouping', 'color_grouping',
                        'fuel_grouping', 'transmission_grouping', 'purchase_accident',
                        'sales_channel_grouping', 'purchase_channel_1_grouping']

X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train[col].astype(str))
    X_test_encoded[col] = le.transform(X_test[col].astype(str))
    label_encoders[col] = le

In [ ]:
X_train_encoded

## 모델 학습

categorical_feature 파라미터: fit() 시 범주형 변수를 명시하여 LightGBM이 최적화된 방식으로 처리하도록 합니다.

In [ ]:
lgbm = LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)

param_distributions = {
    'n_estimators': [100, 200, 300, 500, 800],
    'max_depth': [3, 5, 7, 10, 15, -1],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'num_leaves': [15, 31, 63, 127, 255],
    'min_child_samples': [10, 20, 30, 50],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    'reg_lambda': [0, 0.1, 0.5, 1.0]
}

random_search = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_distributions,
    n_iter=500,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1, # n_jobs=-1 (모든 CPU 사용)일 때는 병렬처리로 인해 각 fold 정보가 실시간으로 출력되지 않습니다.
    random_state=42,
    verbose=2
)

print("RandomizedSearchCV 실행 중...")
random_search.fit(
    X_train_encoded,
    y_train,
    categorical_feature=categorical_features
)

best_model = random_search.best_estimator_
print("\n최적 파라미터:")
print(random_search.best_params_)

## 성능 평가

In [ ]:
y_train_pred = best_model.predict(X_train_encoded)
y_test_pred = best_model.predict(X_test_encoded)

train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_train, y_train_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, y_test_pred)

print("\n" + "="*50)
print("모델 성능 평가")
print("="*50)
print(f"\n[Train Set]")
print(f"  MSE:   {train_mse:.6f}")
print(f"  RMSE:  {train_rmse:.6f}")
print(f"  R²:    {train_r2:.6f}")

print(f"\n[Test Set]")
print(f"  MSE:   {test_mse:.6f}")
print(f"  RMSE:  {test_rmse:.6f}")
print(f"  R²:    {test_r2:.6f}")

## 변수 중요도

In [ ]:
importance_df = pd.DataFrame({
    'Feature': X_train_encoded.columns,
    'Importance': best_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\n" + "="*50)
print("변수 중요도")
print("="*50)
print(importance_df)

# CatBoostRegressor

## 전처리

전처리가 더 간단: LGBM처럼 LabelEncoder도 필요 없고, 범주형 변수를 str 타입으로만 변환하면 CatBoost가 알아서 최적 처리합니다.

In [ ]:
categorical_features = ['manufacturer_grouping', 'car_type_grouping', 'color_grouping',
                        'fuel_grouping', 'transmission_grouping', 'purchase_accident',
                        'sales_channel_grouping', 'purchase_channel_1_grouping']

# 범주형 변수를 str 타입으로만 변환
X_train_cat = X_train.copy()
X_test_cat = X_test.copy()

for col in categorical_features:
    X_train_cat[col] = X_train_cat[col].astype(str)
    X_test_cat[col] = X_test_cat[col].astype(str)

# 범주형 변수의 인덱스 찾기
cat_features_idx = [X_train_cat.columns.get_loc(col) for col in categorical_features]

## 모델 학습

cat_features 파라미터: 범주형 변수의 인덱스를 지정하면 CatBoost가 내부적으로 Target Encoding 등을 자동 적용합니다

In [ ]:
catboost = CatBoostRegressor(
    random_state=42,
    verbose=0,
    cat_features=cat_features_idx  # 범주형 변수 인덱스 지정
)

param_distributions = {
    'iterations': [100, 200, 300, 500, 800],
    'depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'border_count': [32, 64, 128, 255],
    'bagging_temperature': [0, 0.5, 1.0],
    'random_strength': [0, 1, 2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]
}

random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_distributions,
    n_iter=500,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

print("RandomizedSearchCV 실행 중...")
random_search.fit(X_train_cat, y_train)

best_model = random_search.best_estimator_
print("\n최적 파라미터:")
print(random_search.best_params_)

## 성능 평가

In [ ]:
y_train_pred = best_model.predict(X_train_cat)
y_test_pred = best_model.predict(X_test_cat)

train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_train, y_train_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, y_test_pred)

print("\n" + "="*50)
print("모델 성능 평가")
print("="*50)
print(f"\n[Train Set]")
print(f"  MSE:   {train_mse:.6f}")
print(f"  RMSE:  {train_rmse:.6f}")
print(f"  R²:    {train_r2:.6f}")

print(f"\n[Test Set]")
print(f"  MSE:   {test_mse:.6f}")
print(f"  RMSE:  {test_rmse:.6f}")
print(f"  R²:    {test_r2:.6f}")

## 변수 중요도

In [ ]:
importance_df = pd.DataFrame({
    'Feature': X_train_cat.columns,
    'Importance': best_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\n" + "="*50)
print("변수 중요도")
print("="*50)
print(importance_df)

# CatBoostRegressor2

## 전처리

In [ ]:
categorical_features = ['manufacturer_grouping', 'car_type_grouping', 'color_grouping',
                        'fuel_grouping', 'transmission_grouping', 'purchase_accident',
                        'sales_channel_grouping', 'purchase_channel_1_grouping']

X_train_cat = X_train.copy()
X_test_cat = X_test.copy()

for col in categorical_features:
    X_train_cat[col] = X_train_cat[col].astype(str)
    X_test_cat[col] = X_test_cat[col].astype(str)

cat_features_idx = [X_train_cat.columns.get_loc(col) for col in categorical_features]

## 모델 학습

In [ ]:
# catboost = CatBoostRegressor(
#     random_state=42,
#     verbose=0,
#     cat_features=cat_features_idx
# )

# # border_count 제거 (max_bin만 사용)
# param_distributions = {
#     'iterations': [500, 800, 1000, 1500, 2000],
#     'depth': [4, 5, 6, 7, 8, 9, 10],
#     'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1, 0.15],
#     'l2_leaf_reg': [1, 3, 5, 7, 9, 11, 15],
#     'random_strength': [0, 0.5, 1, 1.5, 2],

#     # max_bin만 사용 (border_count와 동일한 역할)
#     'max_bin': [32, 64, 128, 254, 512],

#     # bootstrap 관련 파라미터
#     'bootstrap_type': ['Bayesian', 'Bernoulli', 'MVS'],
#     'bagging_temperature': [0, 0.3, 0.5, 0.7, 1.0],
#     'subsample': [0.7, 0.8, 0.85, 0.9, 0.95, 1.0],

#     # 추가 파라미터
#     'min_data_in_leaf': [1, 5, 10, 20, 30],
#     'one_hot_max_size': [2, 5, 10],
#     'rsm': [0.7, 0.8, 0.9, 1.0],
#     'leaf_estimation_iterations': [1, 5, 10]
# }

# random_search = RandomizedSearchCV(
#     estimator=catboost,
#     param_distributions=param_distributions,
#     n_iter=500,
#     cv=5,
#     scoring='neg_mean_squared_error',
#     n_jobs=-1,
#     random_state=42,
#     verbose=2
# )

# print("RandomizedSearchCV 실행 중...")
# random_search.fit(X_train_cat, y_train)

# best_model = random_search.best_estimator_
# print("\n최적 파라미터:")
# print(random_search.best_params_)

In [ ]:
# 1. RandomizedSearchCV로 최적 파라미터 찾기
catboost = CatBoostRegressor(
    random_state=42,
    verbose=0,
    cat_features=cat_features_idx
)

param_distributions = {
    'iterations': [500, 800, 1000, 1500, 2000],
    'depth': [4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1, 0.15],
    'l2_leaf_reg': [1, 3, 5, 7, 9, 11, 15],
    'random_strength': [0, 0.5, 1, 1.5, 2],
    'max_bin': [32, 64, 128, 254, 512],
    'bootstrap_type': ['Bayesian', 'Bernoulli', 'MVS'],
    'bagging_temperature': [0, 0.3, 0.5, 0.7, 1.0],
    'subsample': [0.7, 0.8, 0.85, 0.9, 0.95, 1.0],
    'min_data_in_leaf': [1, 5, 10, 20, 30],
    'one_hot_max_size': [2, 5, 10],
    'rsm': [0.7, 0.8, 0.9, 1.0],
    'leaf_estimation_iterations': [1, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_distributions,
    n_iter=500,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

print("RandomizedSearchCV 실행 중...")
random_search.fit(X_train_cat, y_train)

best_model = random_search.best_estimator_
print("\n최적 파라미터:")
print(random_search.best_params_)

# 2. 최적 모델로 다시 학습 (학습 과정 기록하면서)
print("\n학습 과정 시각화를 위해 최적 모델 재학습 중...")

final_model = CatBoostRegressor(
    **random_search.best_params_,
    random_state=42,
    cat_features=cat_features_idx,
    verbose=0
)

# eval_set으로 검증 데이터 전달
final_model.fit(
    X_train_cat, y_train,
    eval_set=(X_test_cat, y_test),
    verbose=False
)

# 3. 학습 기록 가져오기
train_loss = final_model.evals_result_['learn']['RMSE']
test_loss = final_model.evals_result_['validation']['RMSE']

# 4. R² 계산 (매 10 epoch마다만 계산 - 속도 향상)
epochs = range(0, len(train_loss), 10)  # 10 epoch마다
train_r2_history = []
test_r2_history = []

print("R² 계산 중...")
for i in epochs:
    y_train_pred = final_model.predict(X_train_cat, ntree_end=i+1)
    y_test_pred = final_model.predict(X_test_cat, ntree_end=i+1)

    train_r2_history.append(r2_score(y_train, y_train_pred))
    test_r2_history.append(r2_score(y_test, y_test_pred))

# 5. 그래프 그리기
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy (R²) 그래프
ax1.plot(list(epochs), train_r2_history, label='Train', color='#1f77b4', linewidth=2)
ax1.plot(list(epochs), test_r2_history, label='Test', color='#ff7f0e', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy (R²)', fontsize=12)
ax1.set_title('Accuracy', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0.85, 1.0])  # R² 범위 조정

# Loss (RMSE) 그래프
ax2.plot(train_loss, label='Train', color='#1f77b4', linewidth=2)
ax2.plot(test_loss, label='Test', color='#ff7f0e', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss (RMSE)', fontsize=12)
ax2.set_title('Loss', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 6. 최종 성능 평가
y_train_pred = final_model.predict(X_train_cat)
y_test_pred = final_model.predict(X_test_cat)

train_r2_final = r2_score(y_train, y_train_pred)
test_r2_final = r2_score(y_test, y_test_pred)

print("\n" + "="*50)
print("최종 모델 성능 평가")
print("="*50)
print(f"\n[Train Set]")
print(f"  R²:    {train_r2_final:.3f} ({train_r2_final*100:.1f}%)")

print(f"\n[Test Set]")
print(f"  R²:    {test_r2_final:.3f} ({test_r2_final*100:.1f}%)")
print("="*50)

# best_model을 final_model로 업데이트
best_model = final_model

## 성능 평가

In [ ]:
# y_train_pred = best_model.predict(X_train_cat)
# y_test_pred = best_model.predict(X_test_cat)

# train_mse = mean_squared_error(y_train, y_train_pred)
# train_rmse = np.sqrt(train_mse)
# train_r2 = r2_score(y_train, y_train_pred)

# test_mse = mean_squared_error(y_test, y_test_pred)
# test_rmse = np.sqrt(test_mse)
# test_r2 = r2_score(y_test, y_test_pred)

# print("\n" + "="*50)
# print("모델 성능 평가")
# print("="*50)
# print(f"\n[Train Set]")
# print(f"  MSE:   {train_mse:.3f}")
# print(f"  RMSE:  {train_rmse:.3f}")
# print(f"  R²:    {train_r2:.3f}")

# print(f"\n[Test Set]")
# print(f"  MSE:   {test_mse:.3f}")
# print(f"  RMSE:  {test_rmse:.3f}")
# print(f"  R²:    {test_r2:.3f}")

In [ ]:
y_train_pred = best_model.predict(X_train_cat)
y_test_pred = best_model.predict(X_test_cat)

train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_train, y_train_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, y_test_pred)

print("\n" + "="*50)
print("모델 성능 평가")
print("="*50)
print(f"\n[Train Set]")
print(f"  MSE:   {train_mse:.3f}")
print(f"  RMSE:  {train_rmse:.3f}")
print(f"  R²:    {train_r2:.3f}")

print(f"\n[Test Set]")
print(f"  MSE:   {test_mse:.3f}")
print(f"  RMSE:  {test_rmse:.3f}")
print(f"  R²:    {test_r2:.3f}")

## 변수 중요도

In [ ]:
# importance_df = pd.DataFrame({
#     'Feature': X_train_cat.columns,
#     'Importance': best_model.feature_importances_
# }).sort_values(by='Importance', ascending=False)

# print("\n" + "="*50)
# print("변수 중요도")
# print("="*50)
# print(importance_df)

In [ ]:
importance_df = pd.DataFrame({
    'Feature': X_train_cat.columns,
    'Importance': best_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\n" + "="*50)
print("변수 중요도")
print("="*50)
print(importance_df)

# CatBoostRegressor3

## 전처리

피처 엔지니어링 - 기존 데이터를 조합/변환해서 새로운 변수를 만드는 작업

<파생 변수 생성, 변수 변환, 변수 선택, 변수 결합, 차원 축소>

In [ ]:
categorical_features = ['manufacturer_grouping', 'car_type_grouping', 'color_grouping',
                        'fuel_grouping', 'transmission_grouping', 'purchase_accident',
                        'sales_channel_grouping', 'purchase_channel_1_grouping']

X_train_cat = X_train.copy()
X_test_cat = X_test.copy()

# 범주형 변수 str 변환
for col in categorical_features:
    X_train_cat[col] = X_train_cat[col].astype(str)
    X_test_cat[col] = X_test_cat[col].astype(str)

# 파생 변수 추가
X_train_cat['mileage_per_year'] = X_train_cat['mileage'] / (X_train_cat['usedyear'] + 1)
X_test_cat['mileage_per_year'] = X_test_cat['mileage'] / (X_test_cat['usedyear'] + 1)

cat_features_idx = [X_train_cat.columns.get_loc(col) for col in categorical_features]

## 모델 학습

In [ ]:
catboost = CatBoostRegressor(
    random_state=42,
    verbose=0,
    cat_features=cat_features_idx
)

# border_count 제거 (max_bin만 사용)
param_distributions = {
    'iterations': [500, 800, 1000, 1500, 2000],
    'depth': [4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1, 0.15],
    'l2_leaf_reg': [1, 3, 5, 7, 9, 11, 15],
    'random_strength': [0, 0.5, 1, 1.5, 2],

    # max_bin만 사용 (border_count와 동일한 역할)
    'max_bin': [32, 64, 128, 254, 512],

    # bootstrap 관련 파라미터
    'bootstrap_type': ['Bayesian', 'Bernoulli', 'MVS'],
    'bagging_temperature': [0, 0.3, 0.5, 0.7, 1.0],
    'subsample': [0.7, 0.8, 0.85, 0.9, 0.95, 1.0],

    # 추가 파라미터
    'min_data_in_leaf': [1, 5, 10, 20, 30],
    'one_hot_max_size': [2, 5, 10],
    'rsm': [0.7, 0.8, 0.9, 1.0],
    'leaf_estimation_iterations': [1, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_distributions,
    n_iter=500,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

print("RandomizedSearchCV 실행 중...")
random_search.fit(X_train_cat, y_train)

best_model = random_search.best_estimator_
print("\n최적 파라미터:")
print(random_search.best_params_)

## 성능 평가

In [ ]:
y_train_pred = best_model.predict(X_train_cat)
y_test_pred = best_model.predict(X_test_cat)

train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_train, y_train_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, y_test_pred)

print("\n" + "="*50)
print("모델 성능 평가")
print("="*50)
print(f"\n[Train Set]")
print(f"  MSE:   {train_mse:.3f}")
print(f"  RMSE:  {train_rmse:.3f}")
print(f"  R²:    {train_r2:.3f}")

print(f"\n[Test Set]")
print(f"  MSE:   {test_mse:.3f}")
print(f"  RMSE:  {test_rmse:.3f}")
print(f"  R²:    {test_r2:.3f}")

## 변수 중요도

In [ ]:
importance_df = pd.DataFrame({
    'Feature': X_train_cat.columns,
    'Importance': best_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\n" + "="*50)
print("변수 중요도")
print("="*50)
print(importance_df)

# GradiantBoosting

## 전처리

전처리: LightGBM, CatBoost와 달리 범주형 변수를 자동 처리하지 못해서 LabelEncoding 필수입니다.

In [ ]:
categorical_features = ['manufacturer_grouping', 'car_type_grouping', 'color_grouping',
                        'fuel_grouping', 'transmission_grouping', 'purchase_accident',
                        'sales_channel_grouping', 'purchase_channel_1_grouping']

X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train[col].astype(str))
    X_test_encoded[col] = le.transform(X_test[col].astype(str))
    label_encoders[col] = le

## 모델 학습

sklearn 기본 모델: 별도 라이브러리 설치 없이 사용 가능하지만, LightGBM/CatBoost보다 속도가 느립니다.

In [ ]:
gb = GradientBoostingRegressor(random_state=42)

param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'max_features': ['sqrt', 'log2', None, 0.5, 0.8],
    'loss': ['squared_error', 'huber', 'absolute_error']
}

random_search = RandomizedSearchCV(
    estimator=gb,
    param_distributions=param_distributions,
    n_iter=500,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

print("RandomizedSearchCV 실행 중...")
random_search.fit(X_train_encoded, y_train)

best_model = random_search.best_estimator_
print("\n최적 파라미터:")
print(random_search.best_params_)

## 성능 평가

In [ ]:
y_train_pred = best_model.predict(X_train_encoded)
y_test_pred = best_model.predict(X_test_encoded)

train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_train, y_train_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, y_test_pred)

print("\n" + "="*50)
print("모델 성능 평가")
print("="*50)
print(f"\n[Train Set]")
print(f"  MSE:   {train_mse:.6f}")
print(f"  RMSE:  {train_rmse:.6f}")
print(f"  R²:    {train_r2:.6f}")

print(f"\n[Test Set]")
print(f"  MSE:   {test_mse:.6f}")
print(f"  RMSE:  {test_rmse:.6f}")
print(f"  R²:    {test_r2:.6f}")

## 변수 중요도

In [ ]:
importance_df = pd.DataFrame({
    'Feature': X_train_encoded.columns,
    'Importance': best_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\n" + "="*50)
print("변수 중요도")
print("="*50)
print(importance_df)